# Brightway 2.5 demonstration

## Installation

* Install [conda](https://docs.conda.io/en/latest/) or miniconda. We use conda as it has a [very fast sparse library](https://www.pardiso-project.org/).
* Create a new conda environment using conda or (better) [mamba](https://mamba-framework.readthedocs.io/en/latest/):

```
    mamba create -y -n bw25test -c conda-forge -c cmutel -c bsteubing -c haasad -c pascallesage pypardiso python=3.8 fs scipy numpy pandas stats_arrays appdirs pip
```

* Activate this environment following the instructions for your OS

* In your `bw25test` environment, install the new development libraries directly from github:

```
    pip install https://github.com/brightway-lca/bw_processing/archive/master.zip
    pip install https://github.com/brightway-lca/matrix_utils/archive/main.zip
    pip install https://github.com/brightway-lca/brightway2-calc/archive/master.zip
```

## Notebook setup

Import new libraries ([bw_processing](https://github.com/brightway-lca/bw_processing) and [matrix_utils](https://github.com/brightway-lca/matrix_utils))

In [1]:
import numpy as np
import bw_processing as bwp
from bw2calc.lca import LCA

Create a "data package" - a set of data resources used to construct matrices, with metadata on provenance, licensing, etc. Based on the Open Knowledge Foundation's [Data Package standard](https://specs.frictionlessdata.io/data-package/).

Because we use [PyFilesystem2](https://docs.pyfilesystem.org/en/latest/), these files can be stored on many logical or virtual filesystems. In this case, we store data in memory.

In [2]:
dp = bwp.create_datapackage()

In memory is the default, we could also make it explicit:

```
from fs.memoryfs import MemoryFS
dp = bwp.create_datapackage(MemoryFS())
```

We can also store data on network drives, via FTP, on cloud platforms, etc. etc. There is a lot of potential here!

Add data. Each matrix (in this case) is constructed by one simple resource. We have the ability, however, to support more complex resource types :)

In [3]:
dp.add_persistent_vector(
    matrix="technosphere_matrix",
    indices_array=np.array(
        [(100, 100), (101, 101), (102, 102), (103, 103),  # Production
         (100, 101), (101, 102), (101, 103), (102, 103)], # Inputs
        dtype=bwp.INDICES_DTYPE  # Means first element is row, second is column
    ),
    flip_array=np.array([
        False, False, False, False, # Production
        True, True, True, True      # Inputs
    ]),
    data_array=np.array([
        1, 1, 1, 1,  # Production
        2, 4, 8, 16  # Inputs
    ]),
)
dp.add_persistent_vector(
    matrix="biosphere_matrix",
    indices_array=np.array(
        [(200, 100), (200, 101), (200, 102), (200, 103), (201, 100), (201, 102)], dtype=bwp.INDICES_DTYPE
    ),
    data_array=np.arange(6),
)
dp.add_persistent_vector(
    matrix="characterization_matrix",
    indices_array=np.array(
        [(200, 200), (201, 201)], dtype=bwp.INDICES_DTYPE
    ),
    data_array=np.array([1, 10]),
)

Building an LCA is then as simple as the functional unit and the data packages

In [4]:
lca = LCA(demand={103: 1}, data_objs=[dp])
lca.lci()
lca.lcia()
lca.score

6667.0

Let's increase the complexity. Sometimes we need to generate data on the fly. This we can do through "interfaces", Python code that generates data or wraps other data sources. These interfaces follow a very simple [bw_processing standard API](https://github.com/brightway-lca/bw_processing#persistent-versus-dynamic).

Intefaces can be classes:

In [ ]:
class ExampleVectorInterface:
    def __next__(self):
        return np.array([1, 1, 1, 1, 2, 4, 8, 16], dtype=np.float64)

In [ ]:
ex = ExampleVectorInterface()
next(ex)

In [ ]:
for _ in range(10):
    print(next(ex) * _)

Or functions:

In [ ]:
from itertools import repeat
vector_interface = repeat(np.array([1, 1, 1, 1, 2, 4, 8, 16], dtype=np.float64))

Interfaces in action:

In [ ]:
dp = bwp.create_datapackage()
dp.add_dynamic_vector(
    matrix="technosphere_matrix",
    # interface=vector_interface,
    interface=ExampleVectorInterface(),  # <- This is the only part that changed
    indices_array=np.array(
        [(100, 100), (101, 101), (102, 102), (103, 103),  # Production
         (100, 101), (101, 102), (101, 103), (102, 103)], # Inputs
        dtype=bwp.INDICES_DTYPE  # Means first element is row, second is column
    ),
    flip_array=np.array([
        False, False, False, False, # Production
        True, True, True, True      # Inputs
    ]),
)
dp.add_persistent_vector(
    matrix="biosphere_matrix",
    indices_array=np.array(
        [(200, 100), (200, 101), (200, 102), (200, 103), (201, 100), (201, 102)], dtype=bwp.INDICES_DTYPE
    ),
    data_array=np.arange(6),
)
dp.add_persistent_vector(
    matrix="characterization_matrix",
    indices_array=np.array(
        [(200, 200), (201, 201)], dtype=bwp.INDICES_DTYPE
    ),
    data_array=np.array([1, 10]),
)

In [ ]:
lca = LCA(demand={103: 1}, data_objs=[dp])
lca.lci()
lca.lcia()
lca.score

We can even treat the interface as a stochastic data source that overwrite existing values:

In [2]:
dp = bwp.create_datapackage()
dp.add_persistent_vector(
    matrix="technosphere_matrix",
    indices_array=np.array(
        [(100, 100), (101, 101), (102, 102), (103, 103),  # Production
         (100, 101), (101, 102), (101, 103), (102, 103)], # Inputs
        dtype=bwp.INDICES_DTYPE  # Means first element is row, second is column
    ),
    flip_array=np.array([
        False, False, False, False, # Production
        True, True, True, True      # Inputs
    ]),
    data_array=np.array([
        1, 1, 1, 1,  # Production
        2, 4, 8, 16  # Inputs
    ]),
)
dp.add_persistent_vector(
    matrix="biosphere_matrix",
    indices_array=np.array(
        [(200, 100), (200, 101), (200, 102), (200, 103), (201, 100), (201, 102)], dtype=bwp.INDICES_DTYPE
    ),
    data_array=np.arange(6),
)
dp.add_persistent_vector(
    matrix="characterization_matrix",
    indices_array=np.array(
        [(200, 200), (201, 201)], dtype=bwp.INDICES_DTYPE
    ),
    data_array=np.array([1, 10]),
)

In [3]:
class RandomInterface:
    def __next__(self):
        return np.hstack([
            np.random.random() * np.array([8, 16])
        ])

In [10]:
overwriter = bwp.create_datapackage()

In [11]:
overwriter.add_dynamic_vector(
    matrix="technosphere_matrix",
    interface=RandomInterface(),
    indices_array=np.array(
        [(101, 103), (102, 103)],  # Indices of the values that will be overwritten
        dtype=bwp.INDICES_DTYPE   
    ),
    flip_array=np.array([           
        True, True      # Inputs
    ]),
)

Eventually, the `LCA` class itself will support `next()`; until then, we can do:

In [12]:
lca = LCA(demand={103: 1}, data_objs=[dp, overwriter])
lca.lci()
lca.lcia()
for _ in range(10):
    next(lca.technosphere_mm)
    lca.lci()
    lca.lcia()
    print(lca.score)

2366.427285347185
6461.401498656212
419.09603593496814
3698.160444062884
6055.095880088351
278.35517448102536
3505.8067247231616
3408.8570096411413
467.4890721944373
1459.7611652744715
